In [2]:
!pip install python-dotenv
!pip install openai
!pip install SpeechRecognition pyaudio nltk scikit-learn semantic-text-splitter gap-statistic tokenizers transformers 
!pip install --upgrade langchain
!pip install langchain_community
!pip install sentence-transformers
!pip install kneed

In [3]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
import speech_recognition as sr

# Load environment variables from .env file
_ = load_dotenv(find_dotenv())

# Get OpenAI API key from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    raise ValueError("OpenAI API key is not set. Please set the 'OPENAI_API_KEY' environment variable in your .env file.")

# Account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"


In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

recognizer = sr.Recognizer()

# Function to recognize speech from the microphone
def recognize_speech_from_mic():
    with sr.Microphone() as source:
        print("Adjusting for ambient noise... Please wait.")
        recognizer.adjust_for_ambient_noise(source)
        print("Listening for speech...")
        audio = recognizer.listen(source)
        try:
            print("Recognizing speech...")
            # Use the Google Web Speech API for recognition
            text = recognizer.recognize_google(audio)
            print(f"Recognized Text: {text}")
            
            # Save the recognized text to a file
            with open("recognized_speech.txt", "w") as file:
                file.write(text + "\n")
                
        except sr.UnknownValueError:
            print("Sorry, I could not understand the audio.")
        except sr.RequestError as e:
            print(f"Could not request results from Google Web Speech API; {e}")

# recognize_speech_from_mic()

In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
file_path = 'recognized_speech.txt' 
with open(file_path, 'r', encoding='utf-8') as file:
    text_data = file.read()

print("what is the prompt you want to ask the text file?")

def prompt_from_mic():
    with sr.Microphone() as source:
        print("Adjusting for ambient noise... Please wait.")
        recognizer.adjust_for_ambient_noise(source)
        print("Listening for question...")
        audio = recognizer.listen(source)
        try:
            print("Recognizing question...")
            # Use the Google Web Speech API for recognition
            prompt = recognizer.recognize_google(audio)
            print(f"Recognized Text: {prompt}")
        except sr.UnknownValueError:
            print("Sorry, I could not understand the audio.")
        except sr.RequestError as e:
            print(f"Could not request results from Google Web Speech API; {e}")
    return prompt

    
question = prompt_from_mic()
if question:
    # Initialize the ChatOpenAI model
    llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

    # Define the prompt template
    prompt = ChatPromptTemplate.from_template(
        "Question: {question}\n\nText Data: {text_data}"
    )

    # Create the LLMChain
    chain = LLMChain(llm=llm, prompt=prompt)

    # Run the chain with both question and text_data
    result = chain.run({"question": question, "text_data": text_data})

    # Print the result
    print(result)
else:
    print("No question recognized.")